In [145]:
def rol_r(num,shift):
    return (num << shift) | (num >> (64-shift))

def rol_l(num,shift):
    return (num >> shift) | ((num << (64-shift)) & (2**65 - 1))

In [146]:
rol_l(rol_r(32,10),10)

32

In [147]:
x = x ^ int("1275437152437512431354",8)
x = (x << 10) | (x >> (64-10))

a = x & 1229782938247303441
b = x & int("0210421042104210421042",8)
c = x & rol_r(1229782938247303441,2)
d = x & rol_r(int("0210421042104210421042",8),2)

aa = int("0100101000011110",16)
bb = int("2002220020022220",16)
cc = int("4444040044044400",16)
dd = int("8880008080000880",16)

In [148]:
[
    print("{0:064b}".format(x)) for x in [aa,bb,cc,dd]
]

0000000100000000000100000001000000000000000000010001000100010000
0010000000000010001000100000000000100000000000100010001000100000
0100010001000100000001000000000001000100000001000100010000000000
1000100010000000000000001000000010000000000000000000100010000000


[None, None, None, None]

## Reversing by hand

We just go backwards. Earlier, we could note that constants which are compared to $a,b,c,d$ do not share any bits. Therefore, we can first reverse the steps to create $a,b,c,d$. The inverse of _and_ is _and_. Next we fuse $xa,xb,xc,xd$ with and _or_ together. It is likely that the applied masks only extract certain bits. This can later be observed.

In [149]:
aa = int("0100101000011110",16)
bb = int("2002220020022220",16)
cc = int("4444040044044400",16)
dd = int("8880008080000880",16) & (2**65 - 1)

In [150]:
xa = aa & 1229782938247303441 
xb = bb & int("0210421042104210421042",8)
xc = cc & rol_r(1229782938247303441,2)
xd = dd & rol_r(int("0210421042104210421042",8),2)

In [151]:
[
    print("{0:064b}".format(x)) for x in [xa,xb,xc,xd]
]

0000000100000000000100000001000000000000000000010001000100010000
0010000000000010001000100000000000100000000000100010001000100000
0100010001000100000001000000000001000100000001000100010000000000
1000100010000000000000001000000010000000000000000000100010000000


[None, None, None, None]

The outputs reveals that no number shares bits. Therefore we _or_ them together. At last, we revert the bitrolling and xoring.

In [152]:
x = xa | xb | xc | xd    

In [153]:
x

17133441828306517936

In [154]:
x = rol_l(x,10)
x

35469068143446000095

In [155]:
x = x ^ int("1275437152437512431354",8)
x

23299538858865275699

In [156]:
codes = []
for i in range(16):
    code = (x >> 4*i) & 0xF
    print(i,"=>",code)
    codes.append(code)

"".join(str(x) for x in codes)

0 => 3
1 => 3
2 => 3
3 => 3
4 => 3
5 => 1
6 => 9
7 => 5
8 => 5
9 => 2
10 => 7
11 => 9
12 => 8
13 => 5
14 => 3
15 => 4


'3333319552798534'

# Let's do z3

In [1]:
from z3 import *#

def rol(num,shift):
    return (num << shift | (num >> (64-shift)))

ModuleNotFoundError: No module named 'z3'

In [136]:
x,y,z = BitVecs("x y z",64)

a = BitVecVal(int("0100101000011110",16),64)
b = BitVecVal(int("2002220020022220",16),64)
c = BitVecVal(int("4444040044044400",16),64)
d = BitVecVal(int("8880008080000880",16),64) # this should take care of uint64

In [137]:
s = Solver()

In [138]:
s.add(y == z ^ int("1275437152437512431354",8))
s.add(x == RotateLeft(y,10))
s.add(x & 1229782938247303441 == a)
s.add(x & int("0210421042104210421042",8) == b)
s.add(x & rol(1229782938247303441,2) == c)
s.add(x & rol(int("0210421042104210421042",8),2) == d)

In [139]:
s.check()

sat

In [140]:
s.model()

[y = 17022324069736448479,
 x = 17133441828306517936,
 z = 4852794785155724083]